<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/oil_and_gas_Process_with_ngl_stabilizer_and_tex_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neqsim
!pip install pydantic
!pip install -U pandas
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 64.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 2.2.2 which is incompatible.


# Process description
In this notebook we will simulate an advanced top side process inculing a NGL column. Fluid inout will be read from an Eclipse 300 file.

https://openarchive.usn.no/usn-xmlui/handle/11250/2439061

In [3]:
import pandas as pd


# 3. Process model

from pydantic import Field, field_validator
from pydantic.dataclasses import dataclass
from typing import Optional, List
from neqsim import jNeqSim
from neqsim.standards import ISO6976
from functools import cache
from neqsim.thermo import dewt, printFrame, TPflash

@dataclass
class ProcessInput():
    """
    A class to define input parameters for the oil process.
    """
    moleRateHP: List[float] = Field(title="molar rates for 22 components to HP manifold [mol/sec]")
    moleRateLP: List[float] = Field(title="molar rates for 22 components to LP manifold [mol/sec]")
    firstStagePressure: float = Field(ge=0.0, le=100.0, title="Pressure of first stage separator [bara]")
    firstStageTemperature: float = Field(ge=0.0, le=100.0, title="Temperature of first stage separator [C]")
    secondStagePressure: float = Field(ge=0.0, le=100.0, title="Second stage separator pressure [bara]")
    secondStageTemperature: float = Field(ge=0.0, le=100.0, title="Second stage separator temperature [C]")
    thirdStagePressure: float = Field(ge=0.0, le=100.0, title="Third stage separator pressure [bara]")
    fourthStagePressure: float = Field(ge=0.0, le=100.0, title="Fourth stage separator pressure [bara]")
    export_oil_temperature: float = Field(ge=0.0, le=100.0, title="Export oil temperature [C]")
    export_oil_pressure: float = Field(ge=0.0, le=100.0, title="Export oil pressure [bara]")
    dewPointScrubberTemperature: float = Field(ge=-100.0, le=100.0, title="Inlet temperature Dew Point scrubber [C]")
    dewPointScrubberPressure: float = Field(ge=0.0, le=100.0, title="Inlet pressure Dew Point scrubber [bara]")
    inlet_tex_temperature: float = Field(ge=-100.0, le=100.0, title="Inlet temperature TEX [C]")
    inlet_tex_pressure: float = Field(ge=0.0, le=100.0, title="Inlet pressure TEX [bara]")
    outlet_tex_pressure: float = Field(ge=0.0, le=100.0, title="OutleTEX pressure DPU [bara]")
    export_gas_pressure: float = Field(ge=0.0, le=200.0, title="Export gas pressure [bara]")
    export_gas_temperature: float = Field(ge=0.0, le=100.0, title="Export gas temperature [C]")
    gasCoolerTemperature:float = Field(ge=0.0, le=100.0, title="Gas cooler temperature [C]")
    pre_flash_temperature:float = Field(ge=-500.0, le=200.0, title="NGL heater temperature [C]")
    pre_flash_pressure:float = Field(ge=0.0, le=100.0, title="NGL heater pressure [bara]")
    nglColumnBottomPressure:float = Field(ge=0.0, le=200.0, title="NGL bottom pressure [bara]")
    nglColumnTopPressure:float = Field(ge=0.0, le=200.0, title="NGL top pressure [bara]")
    nglColumnBottomTemperature:float = Field(ge=-500.0, le=100.0, title="NGL bottom temperature [C]")
    ngl_routing_to_oil:float = Field(ge=0.0, le=1.0, title="fraction of NGL routed to oil to Sture")

    @field_validator('moleRateHP', 'moleRateLP')
    def check_mole_rates(cls, v):
        if len(v) != 16:
            raise ValueError('moleRateHP and moleRateLP must have 22 components')
        return v

@dataclass
class ProcessOutput:
    """
    A class to define output results from an Oil process simulation.
    """
    mass_balance: Optional[float] = None
    recompressor1_power: Optional[float] = None
    recompressor2_power: Optional[float] = None
    recompressor3_power: Optional[float] = None
    exportcompressor_power: Optional[float] = None
    gasexportflow: Optional[float] = None
    oilexportflow: Optional[float] = None
    nglflow: Optional[float] = None
    oilflow: Optional[float] = None
    export_gas_cooler_duty: Optional[float] = None
    tex_flow: Optional[float] = None
    tex_out_temperature: Optional[float] = None
    tvp9: Optional[float] = None
    tvp30: Optional[float] = None
    tvp_oil: Optional[float] = None
    tvp_ngl: Optional[float] = None
    wi: Optional[float] = None
    gcv_gas: Optional[float] = None
    gas_reldens: Optional[float] = None
    tex_out_temperature: Optional[float] = None
    dew_temperature: Optional[float] = None,
    oilheaterduty: Optional[float] = None,
    nglcolumnduty: Optional[float] = None,
    ngl_column_top_temperature: Optional[float] = None,
    orf: List[float] = None,
    energyincludingethane:Optional[float] = None,
    energyincludingpropane:Optional[float] = None,
    energygasfromhotsep:Optional[float] = None,
    heatedoiltocavernTVP:Optional[float] = None,
    temperature_third_stage:Optional[float] = None,
    temperature_fourth_stage:Optional[float] = None,
    compressor_flow_1st_stage:Optional[float] = None,
    compressor_head_1st_stage:Optional[float] = None,
    compressor_flow_2nd_stage:Optional[float] = None,
    compressor_head_2nd_stage:Optional[float] = None,
    compressor_flow_3rd_stage:Optional[float] = None,
    compressor_head_3rd_stage:Optional[float] = None,
    rvp: Optional[float] = None

def getprocess():
    """
    The method creates a oil process object using neqsim
    """

    wellFluid = jNeqSim.thermo.util.readwrite.EclipseFluidReadWrite.read('wellfluid.e300')
    wellFluid .setMultiPhaseCheck(True)
    LPwellFLuid = wellFluid.clone()

    wellStreamHP = jNeqSim.processSimulation.processEquipment.stream.Stream(wellFluid)
    wellStreamHP.setName("HP well stream")

    LPwellStream = jNeqSim.processSimulation.processEquipment.stream.Stream(LPwellFLuid)
    LPwellStream.setName("LP well stream")

    firstStageSeparator = jNeqSim.processSimulation.processEquipment.separator.ThreePhaseSeparator(wellStreamHP)
    firstStageSeparator.setName("1st stage separator")

    oilvalve1 = jNeqSim.processSimulation.processEquipment.valve.ThrottlingValve(firstStageSeparator.getOilOutStream())
    oilvalve1.setOutletPressure(20.0, 'bara')
    oilvalve1.setName("oil depres valve")

    oilFirstStage =  wellStreamHP.clone()
    oilFirstStage.setName("frist stage oil reflux")
    oilFirstStage.setFlowRate(100.0, 'kg/hr')
    oilFirstStage.setPressure(20.0, 'bara')
    oilFirstStage.setTemperature(30.0, 'C')

    oilFirstStageMixer = jNeqSim.processSimulation.processEquipment.mixer.Mixer()
    oilFirstStageMixer.setName("first stage oil mixer")
    oilFirstStageMixer.addStream(oilvalve1.getOutletStream())
    oilFirstStageMixer.addStream(oilFirstStage)

    oilHeaterFromFirstStage = jNeqSim.processSimulation.processEquipment.heatExchanger.Heater(oilFirstStageMixer.getOutStream())
    oilHeaterFromFirstStage.setName("oil heater second stage")

    secondStageSeparator = jNeqSim.processSimulation.processEquipment.separator.ThreePhaseSeparator(oilHeaterFromFirstStage.getOutStream())
    secondStageSeparator.addStream(LPwellStream)
    secondStageSeparator.setName("2nd stage separator")

    oilSeccondStage =  wellStreamHP.clone()
    oilSeccondStage.setName("seccond stage oil reflux")
    oilSeccondStage.setFlowRate(100.0, 'kg/hr')
    oilSeccondStage.setPressure(6.0, 'bara')
    oilSeccondStage.setTemperature(30.0, 'C')

    valve_oil_from_seccond_stage = jNeqSim.processSimulation.processEquipment.valve.ThrottlingValve(secondStageSeparator.getOilOutStream())
    valve_oil_from_seccond_stage.setOutletPressure(7.0, 'bara')
    valve_oil_from_seccond_stage.setName("valve oil from seccond stage")

    oilSeccondStageMixer = jNeqSim.processSimulation.processEquipment.mixer.Mixer()
    oilSeccondStageMixer.setName("seccond stage oil mixer")
    oilSeccondStageMixer.addStream(valve_oil_from_seccond_stage.getOutletStream())
    oilSeccondStageMixer.addStream(oilSeccondStage)

    thirdStageSeparator = jNeqSim.processSimulation.processEquipment.separator.ThreePhaseSeparator(oilSeccondStageMixer.getOutStream())
    thirdStageSeparator.setName("3rd stage separator")

    valve_oil_from_third_stage = jNeqSim.processSimulation.processEquipment.valve.ThrottlingValve(thirdStageSeparator.getOilOutStream())
    valve_oil_from_third_stage.setOutletPressure(3.0, 'bara')
    valve_oil_from_third_stage.setName("valve oil from third stage")

    oilThirdStage =  wellStreamHP.clone()
    oilThirdStage.setName("third stage oil reflux")
    oilThirdStage.setFlowRate(100.0, 'kg/hr')
    oilThirdStage.setPressure(3.0, 'bara')
    oilThirdStage.setTemperature(30.0, 'C')

    oilThirdStageMixer = jNeqSim.processSimulation.processEquipment.mixer.Mixer()
    oilThirdStageMixer.setName("third stage oil mixer")
    oilThirdStageMixer.addStream(valve_oil_from_third_stage.getOutletStream())
    oilThirdStageMixer.addStream(oilThirdStage)

    fourthStageSeparator = jNeqSim.processSimulation.processEquipment.separator.ThreePhaseSeparator(oilThirdStageMixer.getOutStream())
    fourthStageSeparator.setName("4th stage separator")

    firstStageCooler = jNeqSim.processSimulation.processEquipment.heatExchanger.Cooler(fourthStageSeparator.getGasOutStream())
    firstStageCooler.setName("1st stage cooler")

    firstStageScrubber = jNeqSim.processSimulation.processEquipment.separator.Separator(firstStageCooler.getOutStream())
    firstStageScrubber.setName("1st stage scrubber")

    firststagescrubberpump = jNeqSim.processSimulation.processEquipment.pump.Pump(firstStageScrubber.getLiquidOutStream())
    firststagescrubberpump.setName("1st stage scrubber pump")

    firstStageCompressor = jNeqSim.processSimulation.processEquipment.compressor.Compressor(firstStageScrubber.getGasOutStream())
    firstStageCompressor.setName("1st stage compressor")
    firstStageCompressor.setUsePolytropicCalc(True)
    firstStageCompressor.setPolytropicEfficiency(0.8)

    firststagegasmixer = jNeqSim.processSimulation.processEquipment.mixer.Mixer("first stage mixer")
    firststagegasmixer.addStream(firstStageCompressor.getOutStream())
    firststagegasmixer.addStream(thirdStageSeparator.getGasOutStream())

    firstStageCooler2 = jNeqSim.processSimulation.processEquipment.heatExchanger.Cooler(firststagegasmixer.getOutStream())
    firstStageCooler2.setName("1st stage cooler2")

    firstStageScrubber2 = jNeqSim.processSimulation.processEquipment.separator.Separator(firstStageCooler2.getOutStream())
    firstStageScrubber2.setName("1st stage scrubber2")

    firstStageCompressor2 = jNeqSim.processSimulation.processEquipment.compressor.Compressor(firstStageScrubber2.getGasOutStream())
    firstStageCompressor2.setName("2nd stage compressor")
    firstStageCompressor2.setUsePolytropicCalc(True)
    firstStageCompressor2.setPolytropicEfficiency(0.8)

    secondstagegasmixer = jNeqSim.processSimulation.processEquipment.mixer.Mixer("second Stage mixer")
    secondstagegasmixer.addStream(firstStageCompressor2.getOutStream())
    secondstagegasmixer.addStream(secondStageSeparator.getGasOutStream())

    secondStageCooler = jNeqSim.processSimulation.processEquipment.heatExchanger.Cooler(secondstagegasmixer.getOutStream())
    secondStageCooler.setName("2nd stage cooler")

    secondStageScrubber = jNeqSim.processSimulation.processEquipment.separator.Separator(secondStageCooler.getOutStream())
    secondStageScrubber.setName("2nd stage scrubber")

    secondStageCompressor = jNeqSim.processSimulation.processEquipment.compressor.Compressor(secondStageScrubber.getGasOutStream())
    secondStageCompressor.setName("3rd stage compressor")
    secondStageCompressor.setUsePolytropicCalc(True)
    secondStageCompressor.setPolytropicEfficiency(0.8)

    richGasMixer = jNeqSim.processSimulation.processEquipment.mixer.Mixer("fourth Stage mixer")
    richGasMixer.addStream(secondStageCompressor.getOutStream())
    richGasMixer.addStream(firstStageSeparator.getGasOutStream())

    dewPointControlCooler = jNeqSim.processSimulation.processEquipment.heatExchanger.Cooler(richGasMixer.getOutStream())
    dewPointControlCooler.setName("dew point cooler")

    dewPointScrubber = jNeqSim.processSimulation.processEquipment.separator.Separator(dewPointControlCooler.getOutStream())
    dewPointScrubber.setName("dew point scrubber")

    dewPointControlCooler2 = jNeqSim.processSimulation.processEquipment.heatExchanger.Cooler(dewPointScrubber.getGasOutStream())
    dewPointControlCooler2.setName("dew point cooler 2")

    dewPointScrubber2 = jNeqSim.processSimulation.processEquipment.separator.Separator(dewPointControlCooler2.getOutStream())
    dewPointScrubber2.setName("dew point scrubber 2")

    hpLiqmixer = jNeqSim.processSimulation.processEquipment.mixer.Mixer("MP liq gas mixer");
    hpLiqmixer.addStream(dewPointScrubber.getLiquidOutStream());

    mpLiqmixer = jNeqSim.processSimulation.processEquipment.mixer.Mixer("MP liq gas mixer");
    mpLiqmixer.addStream(secondStageScrubber.getLiquidOutStream());

    lpLiqmixer = jNeqSim.processSimulation.processEquipment.mixer.Mixer("LP liq gas mixer");
    lpLiqmixer.addStream(firststagescrubberpump.getOutletStream());
    lpLiqmixer.addStream(firstStageScrubber2.getLiquidOutStream());

    hpResycle = jNeqSim.processSimulation.processEquipment.util.Recycle("HP liq resycle")
    hpResycle.addStream(hpLiqmixer.getOutStream())
    hpResycle.setOutletStream(oilFirstStage)
    hpResycle.setTolerance(1e-6)

    mpResycle = jNeqSim.processSimulation.processEquipment.util.Recycle("MP liq resycle")
    mpResycle.addStream(mpLiqmixer.getOutStream())
    mpResycle.setOutletStream(oilSeccondStage)
    mpResycle.setTolerance(1e-6)

    lpResycle = jNeqSim.processSimulation.processEquipment.util.Recycle("LP liq resycle")
    lpResycle.addStream(lpLiqmixer.getOutStream())
    lpResycle.setOutletStream(oilThirdStage)
    lpResycle.setTolerance(1e-6)

    turboexpander = jNeqSim.processSimulation.processEquipment.compressor.Compressor(dewPointScrubber2.getGasOutStream())
    turboexpander.setName("TEX")
    turboexpander.setIsentropicEfficiency(0.80)

    DPCUScrubber = jNeqSim.processSimulation.processEquipment.separator.Separator(turboexpander.getOutStream())
    DPCUScrubber.setName("TEX LT scrubber")

    NGLpremixer  = jNeqSim.processSimulation.processEquipment.mixer.Mixer("NGL pre mixer");
    NGLpremixer.addStream(DPCUScrubber.getLiquidOutStream())
    NGLpremixer.addStream(dewPointScrubber2.getLiquidOutStream())

    NGLpreflashheater = jNeqSim.processSimulation.processEquipment.heatExchanger.Heater(NGLpremixer.getOutletStream())
    NGLpreflashheater.setName("NGL preflash heater")

    NGLpreflashsseparator = jNeqSim.processSimulation.processEquipment.separator.Separator(NGLpreflashheater.getOutStream())
    NGLpreflashsseparator.setName("NGL pre flash separator")

    NGLfeedvalve = jNeqSim.processSimulation.processEquipment.valve.ThrottlingValve(NGLpreflashsseparator.getLiquidOutStream())
    NGLfeedvalve.setName("NGL column feed valve")

    NGLcolumn = jNeqSim.processSimulation.processEquipment.distillation.DistillationColumn(5, True, False)
    NGLcolumn.setName('NGL column')
    NGLcolumn.addFeedStream(NGLfeedvalve.getOutletStream(), 5)
    NGLcolumn.getReboiler().setOutTemperature(273.15 + 40.0)
    NGLcolumn.setTopPressure(7.5)
    NGLcolumn.setBottomPressure(7.5)

    secondstagegasmixer.addStream(NGLpreflashsseparator.getGasOutStream())
    firststagegasmixer.addStream(NGLcolumn.getGasOutStream())

    NGLsplitter = jNeqSim.processSimulation.processEquipment.splitter.Splitter(NGLcolumn.getLiquidOutStream())
    NGLsplitter.setName("NGL splitter")
    NGLsplitter.setSplitFactors([0.999, 0.001])

    NGLiqmixer = jNeqSim.processSimulation.processEquipment.mixer.Mixer("NGL mixer");
    NGLiqmixer.addStream(fourthStageSeparator.getOilOutStream());
    NGLiqmixer.addStream(NGLsplitter.getSplitStream(0));
    mpLiqmixer.addStream(NGLsplitter.getSplitStream(1));

    exportoil = jNeqSim.processSimulation.processEquipment.heatExchanger.Heater(NGLiqmixer.getOutStream())
    exportoil.setName("export oil cooler")

    exportoilstream = jNeqSim.processSimulation.processEquipment.stream.Stream(exportoil.getOutStream())
    exportoilstream.setName('export oil')

    exportCompressor1 = jNeqSim.processSimulation.processEquipment.compressor.Compressor(DPCUScrubber.getGasOutStream())
    exportCompressor1.setName("export gas compressor")
    exportCompressor1.setUsePolytropicCalc(True)
    exportCompressor1.setPolytropicEfficiency(0.8857)

    exportGasCooler = jNeqSim.processSimulation.processEquipment.heatExchanger.Cooler(exportCompressor1.getOutStream())
    exportGasCooler.setName("export gas cooler")

    exportGas = jNeqSim.processSimulation.processEquipment.stream.Stream(exportGasCooler.getOutStream())
    exportGas.setName("export gas")

    oilprocess = jNeqSim.processSimulation.processSystem.ProcessSystem()
    oilprocess.add(wellStreamHP)
    oilprocess.add(LPwellStream)
    oilprocess.add(firstStageSeparator)
    oilprocess.add(oilvalve1)
    oilprocess.add(oilFirstStage)
    oilprocess.add(oilFirstStageMixer)
    oilprocess.add(oilHeaterFromFirstStage)
    oilprocess.add(secondStageSeparator)
    oilprocess.add(valve_oil_from_seccond_stage)
    oilprocess.add(oilSeccondStage)
    oilprocess.add(oilSeccondStageMixer)
    oilprocess.add(thirdStageSeparator)
    oilprocess.add(valve_oil_from_third_stage)
    oilprocess.add(oilThirdStage)
    oilprocess.add(oilThirdStageMixer)
    oilprocess.add(fourthStageSeparator)
    oilprocess.add(firstStageCooler)
    oilprocess.add(firstStageScrubber)
    oilprocess.add(firststagescrubberpump)
    oilprocess.add(firstStageCompressor)
    oilprocess.add(firststagegasmixer)
    oilprocess.add(firstStageCooler2)
    oilprocess.add(firstStageScrubber2)
    oilprocess.add(firstStageCompressor2)
    oilprocess.add(secondstagegasmixer)
    oilprocess.add(secondStageCooler)
    oilprocess.add(secondStageScrubber)
    oilprocess.add(secondStageCompressor)
    oilprocess.add(richGasMixer)
    oilprocess.add(dewPointControlCooler)
    oilprocess.add(dewPointScrubber)
    oilprocess.add(dewPointControlCooler2)
    oilprocess.add(dewPointScrubber2)
    oilprocess.add(hpLiqmixer)
    oilprocess.add(mpLiqmixer)
    oilprocess.add(lpLiqmixer)
    oilprocess.add(hpResycle)
    oilprocess.add(mpResycle)
    oilprocess.add(lpResycle)
    oilprocess.add(turboexpander)
    oilprocess.add(DPCUScrubber)
    oilprocess.add(NGLpremixer)
    oilprocess.add(NGLpreflashheater)
    oilprocess.add(NGLpreflashsseparator)
    oilprocess.add(NGLfeedvalve)
    oilprocess.add(NGLcolumn)
    oilprocess.add(NGLsplitter)
    oilprocess.add(NGLiqmixer)
    oilprocess.add(exportoil)
    oilprocess.add(exportoilstream)
    oilprocess.add(exportCompressor1)
    oilprocess.add(exportGasCooler)
    oilprocess.add(exportGas)

    return oilprocess


def updateinput(process, locinput):
    """
    Update process with input parameters. Error handling added for non-existing units.
    """
    # Try accessing each unit and catch any potential AttributeError
    try:
      process.getUnit('HP well stream').getFluid().setMolarFlowRates(locinput.moleRateHP)
      process.getUnit('HP well stream').setTemperature(locinput.firstStageTemperature, "C")
      process.getUnit('HP well stream').setPressure(locinput.firstStagePressure, "bara")
      process.getUnit('LP well stream').getFluid().setMolarFlowRates(locinput.moleRateLP)
      process.getUnit('LP well stream').setTemperature(locinput.secondStageTemperature, "C")
      process.getUnit('LP well stream').setPressure(locinput.secondStagePressure, "bara")
      process.getUnit('oil depres valve').setOutletPressure(locinput.secondStagePressure,'bara')
      process.getUnit('oil heater second stage').setOutTemperature(locinput.secondStageTemperature,'C')
      process.getUnit('oil heater second stage').setOutPressure(locinput.secondStagePressure,'bara')
      process.getUnit('valve oil from seccond stage').setOutletPressure(locinput.thirdStagePressure,'bara')
      process.getUnit('valve oil from third stage').setOutletPressure(locinput.fourthStagePressure,'bara')
      process.getUnit('export oil cooler').setOutTemperature(locinput.export_oil_temperature,'C')
      process.getUnit('export oil cooler').setOutPressure(locinput.export_oil_pressure,'bara')
      process.getUnit('1st stage cooler').setOutTemperature(locinput.gasCoolerTemperature,'C')
      process.getUnit('1st stage compressor').setOutletPressure(locinput.thirdStagePressure)
      process.getUnit('1st stage cooler2').setOutTemperature(locinput.gasCoolerTemperature,'C')
      process.getUnit('2nd stage compressor').setOutletPressure(locinput.secondStagePressure)
      process.getUnit('2nd stage cooler').setOutTemperature(locinput.gasCoolerTemperature,'C')
      process.getUnit('3rd stage compressor').setOutletPressure(locinput.firstStagePressure)
      process.getUnit('dew point cooler').setOutTemperature(locinput.dewPointScrubberTemperature,'C')
      process.getUnit('dew point cooler').setOutPressure(locinput.dewPointScrubberPressure,'bara')
      process.getUnit('dew point cooler 2').setOutTemperature(locinput.inlet_tex_temperature,'C')
      process.getUnit('dew point cooler 2').setOutPressure(locinput.inlet_tex_pressure,'bara')
      process.getUnit('TEX').setOutletPressure(locinput.outlet_tex_pressure,'bara')
      process.getUnit('export gas compressor').setOutletPressure(locinput.export_gas_pressure, 'bara')
      process.getUnit('export gas cooler').setOutTemperature(locinput.export_gas_temperature,'C')
      process.getUnit('NGL preflash heater').setOutPressure(locinput.pre_flash_pressure, 'bara')
      process.getUnit('NGL preflash heater').setOutTemperature(locinput.pre_flash_temperature,'C')
      process.getUnit('NGL column feed valve').setOutletPressure(locinput.nglColumnTopPressure, 'bara')
      process.getUnit('NGL column').getReboiler().setOutTemperature(273.15 + locinput.nglColumnBottomTemperature)
      process.getUnit('NGL column').setBottomPressure(locinput.nglColumnBottomPressure)
      process.getUnit('NGL column').setTopPressure(locinput.nglColumnTopPressure)
      process.getUnit('1st stage scrubber pump').setOutletPressure(locinput.thirdStagePressure)
      process.getUnit('NGL splitter').setSplitFactors([locinput.ngl_routing_to_oil, 1.0-locinput.ngl_routing_to_oil])
    except AttributeError as e:
        print(f"Failed to update unit parameters: {e}")


def getoutput(process):
    # update output
    gas = process.getUnit('export gas').getFluid().clone()
    gas.setTemperature(-35.0, 'C')
    gas.setPressure(50.0, 'barg')
    dewtemp = dewt(gas)-273.15

    referenceTemperatureVolume = "15.0"
    referenceTemperatureCombustion = "25.0"
    numberUnit = 'volume'

    iso6976 = ISO6976(process.getUnit('export gas').getFluid())
    iso6976.setReferenceType(numberUnit)
    iso6976.setVolRefT(float(referenceTemperatureVolume))
    iso6976.setEnergyRefT(float(referenceTemperatureCombustion))
    iso6976.calculate()

    wig = iso6976.getValue("SuperiorWobbeIndex")/1.0e3
    gcvg = iso6976.getValue("SuperiorCalorificValue")/1.0e3
    reldensg = iso6976.getValue("RelativeDensity")

    rf = []
    componentNames = process.getUnit('export oil').getFluid().getComponentNames()
    for name in componentNames:
      total = process.getUnit('export oil').getFluid().getComponent(name).getTotalFlowRate('mole/sec') + process.getUnit('export gas').getFluid().getComponent(name).getTotalFlowRate('mole/sec')
      rf.append(process.getUnit('export oil').getFluid().getComponent(name).getTotalFlowRate('mole/sec')/total)

    oilfeed = process.getUnit('export oil')
    oilfeed.run()
    #printFrame(oilfeed.getFluid())

    cavernfeed = jNeqSim.processSimulation.processEquipment.heatExchanger.Heater(oilfeed)
    cavernfeed.setName("oil cavern feed")
    cavernfeed.setOutPressure(1.2, 'barg')
    cavernfeed.setOutTemperature(9.0, 'C')
    cavernfeed.run()
    #printFrame(cavernfeed.getOutStream().getFluid())

    oil_cavern = jNeqSim.processSimulation.processEquipment.separator.ThreePhaseSeparator(cavernfeed.getOutStream())
    oil_cavern.setName("oil cavern")
    oil_cavern.run()

    oil_HX1 = jNeqSim.processSimulation.processEquipment.heatExchanger.Heater(oil_cavern.getOilOutStream())
    oil_HX1.setName("oil HX1")
    oil_HX1.setOutTemperature(40.0, 'C')
    oil_HX1.setOutPressure(9.0, 'bara')
    oil_HX1.run()

    oil_HX2 = jNeqSim.processSimulation.processEquipment.heatExchanger.Heater(oil_HX1.getOutStream())
    oil_HX2.setName("oil HX2")
    oil_HX2.setOutTemperature(50.0, 'C')
    oil_HX2.run()

    oil_heater1 = jNeqSim.processSimulation.processEquipment.heatExchanger.Heater(oil_HX2.getOutStream())
    oil_heater1.setName("oil hot oil heater 1")
    oil_heater1.setOutTemperature(100.0, 'C')
    oil_heater1.setOutPressure(2.3, 'bara')
    oil_heater1.run()

    heatedoil_separator = jNeqSim.processSimulation.processEquipment.separator.ThreePhaseSeparator(oil_heater1.getOutStream())
    heatedoil_separator.setName("heated oil separator")
    heatedoil_separator.run()

    oiltocavern = heatedoil_separator.getOilOutStream()

    heatedoiltocavernTVP = oiltocavern.getTVP(30.0, 'C', 'bara')

    gasstream = heatedoil_separator.getGasOutStream()
    gasstream.run()

    iso6976 = ISO6976(gasstream.getFluid())
    iso6976.setReferenceType(numberUnit)
    iso6976.setVolRefT(float(referenceTemperatureVolume))
    iso6976.setEnergyRefT(float(referenceTemperatureCombustion))
    iso6976.calculate()

    gasfromhotsepenergy = gasstream.getFlowRate('Sm3/sec') * iso6976.getValue("SuperiorCalorificValue")/1.0e3
    #print('Energy of all gas from hot separator  ',  gasstream.getFlowRate('Sm3/sec') * iso6976.getValue("SuperiorCalorificValue")/1.0e3, ' MW')

    compsplitter = jNeqSim.processSimulation.processEquipment.splitter.ComponentSplitter('splitter' , oilfeed)
    compsplitter.setSplitFactors([1,1,1,1,1.0,0,0,0,0,0,0,0,0,0,0,0])
    compsplitter.run()

    gasstream = compsplitter.getSplitStream(0)
    gasstream.run()
    #printFrame(gasstream.getFluid())

    iso6976 = ISO6976(gasstream.getFluid())
    iso6976.setReferenceType(numberUnit)
    iso6976.setVolRefT(float(referenceTemperatureVolume))
    iso6976.setEnergyRefT(float(referenceTemperatureCombustion))
    iso6976.calculate()

    lightcompenergy = gasstream.getFlowRate('Sm3/sec') * iso6976.getValue("SuperiorCalorificValue")/1.0e3

    compsplitter = jNeqSim.processSimulation.processEquipment.splitter.ComponentSplitter('splitter' , oilfeed)
    compsplitter.setSplitFactors([1,1,1,1,0.0,0,0,0,0,0,0,0,0,0,0,0])
    compsplitter.run()

    gasstream = compsplitter.getSplitStream(0)
    gasstream.run()
    #printFrame(gasstream.getFluid())

    iso6976 = ISO6976(gasstream.getFluid())
    iso6976.setReferenceType(numberUnit)
    iso6976.setVolRefT(float(referenceTemperatureVolume))
    iso6976.setEnergyRefT(float(referenceTemperatureCombustion))
    iso6976.calculate()

    energyincludingethane = gasstream.getFlowRate('Sm3/sec') * iso6976.getValue("SuperiorCalorificValue")/1.0e3

    rvpfluid =process.getUnit('export oil').getFluid().clone()
    rvpfluid.init(0)
    rvpfluid.setMultiPhaseCheck(False)
    rvpfluid.setPressure(1.0, 'bara')
    rvpfluid.setTemperature(37.8, 'C')
    TPflash(rvpfluid)

    standard_ASTM_D6377 = jNeqSim.standards.oilQuality.Standard_ASTM_D6377(rvpfluid)
    standard_ASTM_D6377.setReferenceTemperature(37.8, "C")
    standard_ASTM_D6377.calculate()
    reidvappres = standard_ASTM_D6377.getValue("RVP", "bara")
    #print('Energy of light components including propane  ',  gasstream.getFlowRate('Sm3/sec') * iso6976.getValue("SuperiorCalorificValue")/1.0e3, ' MW')


    outputparam = {
        'mass_balance': (process.getUnit('HP well stream').getFlowRate('kg/hr')+process.getUnit('LP well stream').getFlowRate('kg/hr')-process.getUnit('export gas').getFlowRate('kg/hr')-process.getUnit('export oil').getFlowRate('kg/hr'))/(process.getUnit('HP well stream').getFlowRate('kg/hr')+process.getUnit('LP well stream').getFlowRate('kg/hr'))*100.0,
        'recompressor1_power': process.getUnit('1st stage compressor').getPower()/1e3,
        'export_gas_cooler_duty':process.getUnit('export gas cooler').getDuty()/1e3,
        'recompressor2_power': process.getUnit('2nd stage compressor').getPower()/1e3,
        'recompressor3_power': process.getUnit('3rd stage compressor').getPower()/1e3,
        'exportcompressor_power': process.getUnit('export gas compressor').getPower()/1e3,
        'oilexportflow':process.getUnit('export oil').getFlowRate('idSm3/hr')*24.0,
        'nglflow':process.getUnit('NGL column').getLiquidOutStream().getFlowRate('idSm3/hr')*24.0,
        'ngl_column_top_temperature':process.getUnit('NGL column').getGasOutStream().getTemperature('C'),
        'oilflow':process.getUnit('4th stage separator').getOilOutStream().getFlowRate('idSm3/hr')*24.0,
        'gasexportflow':process.getUnit('export gas').getFlowRate('MSm3/day'),
        'tex_flow': process.getUnit('TEX').getOutletStream().getFlowRate('kg/hr'),
        'tex_out_temperature': process.getUnit('TEX').getOutletStream().getTemperature('C'),
        'tvp30': process.getUnit('export oil').getTVP(30.0, 'C', 'bara'),
        'tvp9': process.getUnit('export oil').getTVP(9.0, 'C', 'bara'),
        'tvp_oil': process.getUnit('4th stage separator').getOilOutStream().getTVP(30.0, 'C', 'bara'),
        'tvp_ngl': process.getUnit('NGL column').getLiquidOutStream().getTVP(30.0, 'C', 'bara'),
        'wi': wig,
        'tex_out_temperature': process.getUnit('TEX').getOutletStream().getTemperature('C'),
        'dew_temperature': dewtemp,
        'gcv_gas': gcvg,
        'gas_reldens': reldensg,
        'orf': rf,
        'oilheaterduty': process.getUnit('oil heater second stage').getDuty()/1e3,
        'nglcolumnduty': process.getUnit('NGL column').getReboiler().getDuty()/1e3,
        'energyincludingethane': energyincludingethane,
        'energyincludingpropane': lightcompenergy,
        'energygasfromhotsep': gasfromhotsepenergy,
        'heatedoiltocavernTVP':heatedoiltocavernTVP,
        'temperature_third_stage': process.getUnit('3rd stage separator').getGasOutStream().getTemperature('C'),
        'temperature_fourth_stage':process.getUnit('4th stage separator').getGasOutStream().getTemperature('C'),
        'compressor_flow_1st_stage':process.getUnit('1st stage compressor').getOutStream().getFlowRate('m3/hr'),
        'compressor_head_1st_stage':process.getUnit('1st stage compressor').getPolytropicHead(),
        'compressor_flow_2nd_stage':process.getUnit('2nd stage compressor').getOutStream().getFlowRate('m3/hr'),
        'compressor_head_2nd_stage':process.getUnit('2nd stage compressor').getPolytropicHead(),
        'compressor_flow_3rd_stage':process.getUnit('3rd stage compressor').getOutStream().getFlowRate('m3/hr'),
        'compressor_head_3rd_stage':process.getUnit('3rd stage compressor').getPolytropicHead(),
        'rvp': reidvappres,
    }

    return outputparam

def run_process_with_timeout(process, input_params, timeout=60, runstep=False):
    """Run the oil process calculation with a specified timeout."""
    updateinput(process=process, locinput=ProcessInput(**input_params))
    process.setRunInSteps(runstep)
    thread = process.runAsThread()
    thread.join(timeout*1000)
    if thread.isAlive():
        print('Process calculation timed out. Consider recreating the process object or adjusting parameters.')
        return None
    return ProcessOutput(**getoutput(process=process))

In [ ]:
oseberg_process = getprocess()

feed = #read flow rate of components for a given year

HP1 = [x * 0.8 for x in feed]
LP1 = [x * 0.2 for x in feed]
inputparam = {
    'moleRateHP': HP1,
    'moleRateLP': LP1,
    'gasCoolerTemperature': 28.0,
    'export_gas_temperature': 55.0,
    'export_gas_pressure': 126.5,
    'firstStagePressure': 62.0,
    'firstStageTemperature': 67.0,
    'secondStagePressure': 20.0,
    'secondStageTemperature': 80.0,
    'thirdStagePressure': 6.0,
    'fourthStagePressure': 2.0,
    'export_oil_temperature': 20.0,
    'export_oil_pressure': 20.8,
    'dewPointScrubberTemperature': 24.0,
    'dewPointScrubberPressure': 60.6,
    'inlet_tex_temperature': -5.0,
    'inlet_tex_pressure': 59.6,
    'outlet_tex_pressure': 50.0,
    'pre_flash_temperature': 0.0,
    'pre_flash_pressure' :20.6,
    'nglColumnTopPressure': 7.3,
    'nglColumnBottomPressure' :7.3,
    'nglColumnBottomTemperature' :75.0,
    'ngl_routing_to_oil': 0.001
}

# Update input parameters for process calculations.
# Note: In case of a process crash, the `oilprocess` object should be recreated to ensure stability for subsequent runs.
# Reusing the `oilprocess` object without unnecessary recreation can enhance numerical performance,
# as new calculations can leverage the solutions from previous runs. However, ensure to manage this carefully to avoid state-related errors.
# The `getprocess()` function is tasked with either creating a new `oilprocess` object or fetching an existing one,
# depending on the design to balance between computational efficiency and error resilience.

runstep=True
for i in range(4):
  #timout parameter has unit seconds
  results = run_process_with_timeout(process=oseberg_process, input_params=inputparam, timeout=60,runstep=runstep)
  #print('Calculation . Results: %s', results.__dict__ if results else 'No results')
  if results and (results.mass_balance > 0.5 and runstep==False):
      print('Calculation failed to converge. Recreating process object...')
      print('Mass balance: ' , results.mass_balance, ' %')
      oilprocess = getprocess()
  #else:
  #    print('Calculation successful. Results: %s', results.__dict__ if results else 'No results')

print(' Calculation successful. Results: %s', results.__dict__ if results else 'No results')